In [1]:
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms as T
import torch.nn as nn

In [2]:
import os

import PIL.Image
import pandas as pd
import scipy.io
import torch
from torch.utils.data import Dataset

In [3]:
random_seed = 1
torch.manual_seed(random_seed)

In [4]:
def int_to_onehot(y, num_labels = 102):
    ary = np.zeros(num_labels)
    ary[y-1] = 1
    return ary

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = scipy.io.loadmat(annotations_file)['labels'][0]
        self.img_list = os.listdir(img_dir)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_list[idx])
        image = PIL.Image.open(img_path).convert('RGB')
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, int_to_onehot(label)

In [6]:
num_classes = 102

resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(512, num_classes)

googlenet = models.googlenet(pretrained=True)
googlenet.fc = nn.Linear(1024, num_classes)

normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [7]:
print(resnet18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
dataset = CustomImageDataset("imagelabels.mat", "102flowers/jpg", transform=transform)

train_set, val_set = torch.utils.data.random_split(dataset, [5733, 2456])

batch_size_train = 64
batch_size_test = 64

train_loader = torch.utils.data.DataLoader(train_set,
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(val_set,
  batch_size=batch_size_train, shuffle=True)

In [9]:
n_epochs = 50
learning_rate = 0.1
log_interval = 10

network = resnet18
optimizer = optim.SGD(network.parameters(), lr=learning_rate)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
        output = network(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.max(1, keepdim=True)[1]
        targetNumber = target.max(1, keepdim=True)[1]
        correct += pred.eq(targetNumber.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  test_acc_torch = 100. * correct / len(test_loader.dataset)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    test_acc_torch))
  return test_acc_torch

In [10]:
test_acc_torch = test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test_acc_torch = test()

C:\Users\hilak\anaconda3\envs\ANN\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 4.7882, Accuracy: 13/2456 (0.53%)

Train Epoch: 1 [0/5733 (0%)]	Loss: 4.829283
Train Epoch: 1 [640/5733 (11%)]	Loss: 3.193860
Train Epoch: 1 [1280/5733 (22%)]	Loss: 1.815580
Train Epoch: 1 [1920/5733 (33%)]	Loss: 1.483085
Train Epoch: 1 [2560/5733 (44%)]	Loss: 0.943570
Train Epoch: 1 [3200/5733 (56%)]	Loss: 0.855164
Train Epoch: 1 [3840/5733 (67%)]	Loss: 0.698113
Train Epoch: 1 [4480/5733 (78%)]	Loss: 0.661347
Train Epoch: 1 [5120/5733 (89%)]	Loss: 0.680575


KeyboardInterrupt: 